In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
a = pd.read_csv("../기상청공모전/data/derivate_data.csv")

In [3]:
a1 = a.iloc[:,1:]

In [4]:
a1 = a1[a1["cog"] != -99.9]
a1 = a1[a1["hdg"] != -99.9]
a1 = a1[a1["hdg"] != -99]

In [6]:
!pip install lightgbm

     ---------------------------------------- 1.0/1.0 MB 9.2 MB/s eta 0:00:00



[notice] A new release of pip available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# LightGBM
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = a1.iloc[:, 2:]  # 세번째 열부터 마지막 열까지 입력값
y = a1.iloc[:, 0]  # 첫번째 열이 정답열

# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM 데이터셋 생성
train_data = lgb.Dataset(X_train, label=y_train)

# LightGBM 모델 학습
params = {'objective': 'binary', 'metric': 'binary_logloss'}
num_rounds = 100  # 반복 횟수 설정

model = lgb.train(params, train_data, num_rounds)

# 테스트 데이터 예측
y_pred = model.predict(X_test)
y_pred_binary = [1 if y >= 0.5 else 0 for y in y_pred]

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred_binary)
print("정확도:", accuracy)

[LightGBM] [Info] Number of positive: 11361, number of negative: 518561
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030381 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2138
[LightGBM] [Info] Number of data points in the train set: 529922, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.021439 -> initscore=-3.820871
[LightGBM] [Info] Start training from score -3.820871
정확도: 0.991281768706456


In [8]:
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred_binary))

0.7795380797862187


In [9]:
from sklearn.metrics import confusion_matrix

# 혼동 행렬 계산
cm = confusion_matrix(y_test, y_pred_binary)

# CSI 계산
tp = cm[1, 1]  # True Positive
tn = cm[0, 0]  # True Negative
fp = cm[0, 1]  # False Positive
fn = cm[1, 0]  # False Negative

csi = tp / (tp + fn + fp)
print("CSI 점수:", csi)

CSI 점수: 0.638723803565843
